In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import json
import MDAnalysis as mda
import nglview as nv

In [12]:
PATH = '../data/SIU'
pockets = json.load(open(os.path.join(PATH, 'pockets_info.json')))
pdb_ids = list(pockets.keys())
len(pdb_ids)

194

In [13]:
i=0

In [14]:
def get_selection(size, center, sim):
    center = np.array(center)
    size = np.array(size)
    # Get atoms
    all_atoms = sim.select_atoms("all")
    coords = all_atoms.positions
    # Create boolean mask for atoms inside the box
    mask = (np.abs(coords - center.reshape(1,3)) < size/2).all(axis=1)
    pocket_atoms = all_atoms[mask]
    pocket_residues = pocket_atoms.residues

    # Use segid instead of chainID
    pocket_selection = " or ".join([f"resi {res.resid}" for res in pocket_residues])
    return pocket_selection


In [15]:
def get_protein_and_pocket_fpocket(pdb_id):
    u = mda.Universe(os.path.join(PATH, "pdb_files", f"{pdb_id}.pdb"))
    pocket_u = mda.Universe(os.path.join(PATH, "pockets", f"{pdb_id}_out", "pockets", "pocket1_atm.pdb"))
    print(len(pocket_u.residues))
    pocket_selection = " or ".join([f"resi {res.resid}" for res in pocket_u.residues])
    protein_atoms = u.select_atoms("protein")

    # Create a new universe with only protein atoms
    u_protein = mda.Merge(protein_atoms)
    u_protein.atoms.translate(-u_protein.atoms.center_of_mass())  # Optional: center the protein
    view = nv.show_mdanalysis(u_protein)
    view.add_representation("surface", colorScheme="hydrophobicity", selection=pocket_selection)

    return view, pocket_selection

def get_protein_and_vina_box(pdb_id):
    u = mda.Universe(os.path.join(PATH, "pdb_files", f"{pdb_id}.pdb"))
    protein_atoms = u.select_atoms("protein")
    pocket_info = pockets[pdb_id]
    u_protein = mda.Merge(protein_atoms)

    pocket_selection = get_selection(
        np.array(pocket_info["size"]), pocket_info["center"], u_protein
    )
    u_protein.atoms.translate(-u_protein.atoms.center_of_mass())  # Optional: center the protein

    view = nv.show_mdanalysis(u_protein)
    view.add_representation("surface", colorScheme="hydrophobicity", selection=pocket_selection)

    return view, pocket_selection

In [25]:
i = 0

In [70]:
pdb_id = pdb_ids[i]
i+=1
view, pocket_selection = get_protein_and_pocket_fpocket(pdb_id)
view

14


NGLWidget()

In [71]:
view.download_image(f"{pdb_id}_pocket.png", factor=2, trim=True)

In [72]:
view, pocket_selection2 = get_protein_and_vina_box(pdb_id)
view

NGLWidget()

In [73]:
view.download_image(f"{pdb_id}_vina_box.png", factor=2, trim=True)


In [ ]:
key